In [1]:
import numpy as np
import pandas as pd
from math import floor
from dataloaders import *
from utils import *
from EDA import *
import config

In [2]:
dataset_dir = config.dataset_dir

for subject_id in range(10):
    subject_id += 1
    
    for sic_candidate in range(30):
        sic_candidate += 24
        
        print('\nSubject ID:',subject_id,'\nsIC:',sic_candidate)

        bold, _ = data_import(dataset    = dataset_dir,
                              subject_id = subject_id,
                              subfolder  = 'pre_bold\\pre_bold_preprocessed.ica',
                              data_name  = ['filtered_func_data'])

        fmri_data = bold['filtered_func_data'].get_fdata()

        sica, _ = data_import(dataset    = dataset_dir,
                              subject_id = subject_id,
                              subfolder  = 'pre_bold\\pre_bold_preprocessed.ica\\filtered_func_data.ica',
                              data_name  = ['mask','melodic_IC'])

        selected_ic = ic_extract(sica['melodic_IC'],sic_candidate)

        _, selected_ic_cluster = ic_browse(selected_ic = selected_ic,
                                           criterion   = selected_ic >= 3.1,
                                           affine      = sica['melodic_IC'].affine,
                                           show        = False)

        selected_ic_cluster = morphological_operation(selected_ic_cluster)

        max_value, _ = ic_browse(selected_ic = selected_ic,
                                 criterion   = selected_ic >= selected_ic.max(),
                                 affine      = sica['melodic_IC'].affine,
                                 show        = False)
        
        max_value_fmri = fmri_data[max_value["Max_index"][0]]
        max_area_fmri = mean_fmri_in_neighborhood(fmri_data, max_value["Max_index"][0], 3)
        selected_ic_mask = mask_cc_around_coordinate(selected_ic_cluster, max_value["Max_index"][0])
        downsample_factor = floor((np.sum(selected_ic_mask)/32)**(1/3))
        # temporal_loading and frequency_loading in melodic_mix and melodic_FTmix
        
        try:
            R_outside_inside = outside_inside_ratio(selected_ic, sica['mask'])
        except Exception as e:
            R_outside_inside = float('nan')
        try:
            max_power_freq = max_power_frequency(max_area_fmri, tr=3.0)
        except Exception as e:
            max_power_freq = float('nan')
        try:
            LI_result = lateralization_index(selected_ic)
        except Exception as e:
            LI_result = {'Lateralization Index':float('nan')}
        try:
            BLI_result = binary_lateralization(selected_ic_cluster)
        except Exception as e:
            BLI_result = {'Lateralization Strength':float('nan'),
                          'Side':float('nan')}
        try:
            connectivity_matrix = cluster_connectivity_ds(fmri_data, selected_ic_mask, downsample_factor)
        except Exception as e:
            connectivity_matrix = float('nan')
        try:
            central_network_strength = voxel_network_strength(fmri_data, selected_ic_mask, max_value["Max_index"][0])
        except Exception as e:
            central_network_strength = float('nan')
        try:
            random_matrices = generate_random_matrices(connectivity_matrix, num_random_matrices=500)
        except Exception as e:
            random_matrices = float('nan')
        try:
            TM_result = compute_topological_measures(connectivity_matrix, random_matrices)
        except Exception as e:
            TM_result = {'Clustering Coefficient':float('nan'),
                         'Characteristic Path Length':float('nan'),
                         'Local Efficiency':float('nan'),
                         'Connectivity Strength':float('nan'),
                         'Connectivity Diversity':float('nan')}

        sIC_Features = {
            'Subject ID': subject_id,
            'sIC Number': sic_candidate,
            'sIC Center': max_value['MNI_coords'],
            'R_o/i': R_outside_inside,
            'Max Power Frequency': max_power_freq,
            'Lateralization Index': LI_result['Lateralization Index'],
            'Lateralization Strength': BLI_result['Lateralization Strength'],
            'Lateralization Side': BLI_result['Side'],
            'Centeral Network': central_network_strength,
            'Clustering Coefficient': TM_result['Clustering Coefficient'],
            'Characteristic Path Length': TM_result['Characteristic Path Length'],
            'Local Efficiency': TM_result['Local Efficiency'],
            'Connectivity Strength': TM_result['Connectivity Strength'],
            'Connectivity Diversity': TM_result['Connectivity Diversity'],  
        }

        df = pd.DataFrame(sIC_Features, index=[0])
        results_path = f'results/Sub{subject_id}_sIC_Features.csv'

        if os.path.isfile(results_path):
            df.to_csv(results_path, mode='a', index=False, header=False)
        else:
            df.to_csv(results_path, index=False, header=True)


Subject ID: 1 
sIC: 24

selected data:
D:\Academic\Datasets\Epilepsy_rsfMRI_Fallahi\Sub001\pre_bold\pre_bold_preprocessed.ica\filtered_func_data.nii.gz  >> Shape: (91, 109, 91, 320)

selected data:
D:\Academic\Datasets\Epilepsy_rsfMRI_Fallahi\Sub001\pre_bold\pre_bold_preprocessed.ica\filtered_func_data.ica\mask.nii.gz  >> Shape: (91, 109, 91)
D:\Academic\Datasets\Epilepsy_rsfMRI_Fallahi\Sub001\pre_bold\pre_bold_preprocessed.ica\filtered_func_data.ica\melodic_IC.nii.gz  >> Shape: (91, 109, 91, 30)

Subject ID: 1 
sIC: 25

selected data:
D:\Academic\Datasets\Epilepsy_rsfMRI_Fallahi\Sub001\pre_bold\pre_bold_preprocessed.ica\filtered_func_data.nii.gz  >> Shape: (91, 109, 91, 320)

selected data:
D:\Academic\Datasets\Epilepsy_rsfMRI_Fallahi\Sub001\pre_bold\pre_bold_preprocessed.ica\filtered_func_data.ica\mask.nii.gz  >> Shape: (91, 109, 91)
D:\Academic\Datasets\Epilepsy_rsfMRI_Fallahi\Sub001\pre_bold\pre_bold_preprocessed.ica\filtered_func_data.ica\melodic_IC.nii.gz  >> Shape: (91, 109, 91